In [ ]:
from pathlib import Path
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

from fastai.vision.all import load_learner
import numpy as np
from PIL import Image
from IPython.display import display

np.int = np.int32 # Need to add this to make preset model work

In [ ]:
def add_mask2(source: Image.Image, truth: np.ndarray, pred: np.ndarray) -> Image.Image:
    """
    Given source black and white image, true and predicted mask this function returns image
    with areas colored as following:
    - Green - annotation only
    - Red - prediction only
    - Yellow - overlap
    """
    source = source.convert('RGBA')
    M = np.zeros((*truth.shape, 4), dtype=np.uint8)
    M[:, :, 1] = truth[:, :] * 255
    M[:, :, 0] = pred[:, :] * 255
    M[:, :, 3] = ((truth > 0) | (pred > 0)) * 75
    
    mask = Image.fromarray(M, 'RGBA')
    return Image.alpha_composite(source, mask)

In [ ]:
def image2segmentation_path(imgpath: Path) -> Path:
    return Path(str(imgpath).replace("inputs", "segmentations"))

def pixels2area(n: int) -> float:
    """Converts number of pixels into area in um^2"""
    return n * 0.023 * 0.023

def area2mass(A: float) -> float:
    """Converts area in um^2 into mass in mg"""
    return 0.197 * (A ** 1.38)

In [ ]:
model = load_learner("models/learner.pkl")  # File models/learner.pkl
model.load("model_resnet34")  # File models/model_resnet34.pth
model.dls.device = 'cpu'

In [ ]:
# imgs = sorted(list(Path("images/example_inputs_orig").glob("*.bmp")))
imgs = sorted(list(Path("images/example_inputs_lowres").glob("*.bmp")))
# imgs = sorted(list(Path("images/example_inputs_lowres2").glob("*.bmp")))
# imgs

In [ ]:
for pimg in imgs:
    im = Image.open(pimg)
    p_segmentation = image2segmentation_path(pimg)

    mask, *_ = model.predict(pimg)
    im_pred_mask = mask.numpy()

    if p_segmentation is not None:
        im_truth = np.array(Image.open(p_segmentation).convert("L"))
    else:
        im_truth = np.zeros_like(im_pred_mask)

    img_with_masks = add_mask2(im, im_truth, im_pred_mask)

    lipid_annotated = area2mass(pixels2area((im_truth > 0).sum()))
    lipid_predicted = area2mass(pixels2area((im_pred_mask > 0.5).sum()))
    print("************************")
    print(pimg)
    print(f"Lipid annotation: {lipid_annotated:.5}mg")
    print(f"Lipid prediction: {lipid_predicted:.5}mg")
    # Green - annotation
    # Red - prediction
    # Yellow - overlap
    display(img_with_masks.convert("RGB"))